<a href="https://colab.research.google.com/github/GiuliaSolinas/AIAgents/blob/main/Copy_of_CrewAI_Flows_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 **CrewAI Flow: Dynamic Content Creation Router**

## Project Overview

This notebook demonstrates a **CrewAI Flow** that dynamically routes content creation tasks to specialized crew of agents based on user input. This workflow showcases CrewAI's advanced Flow architecture with **state persistence**, **event-driven routing**, and **multi-crew orchestration**.

### 🎯 What you'll learn

- **Flow Architecture**: Event-driven workflows with `@start`, `@router`, and `@listen` decorators
- **State Management**: Persistent state across workflow execution with `@persist`
- **Dynamic Routing**: Intelligent crew selection based on content type
- **Multi-Crew Orchestration**: Coordinating specialized teams for different content formats

### 🌊 Flow Steps:
1. **Input Collection**: Gather URL and desired content type from user
2. **Dynamic Routing**: Route to appropriate specialized crew based on content type
3. **Content Processing**: Execute specialized content creation with dedicated crews
4. **Output Finalization**: Return polished, ready-to-use content

First we'll start by installing the required packages.

In [ ]:
%pip -q install -U crewai==0.177.0 crewai_tools

Set the API Keys needed:

- `OPENAI_API_KEY` or any LLM provider you prefer
- `SERPER_API_KEY` they have a generous free tier you can use. Get your key at https://serper.dev

In [ ]:
from google.colab import userdata
import os

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
SERPER_API_KEY = userdata.get('SERPER_API_KEY')

if OPENAI_API_KEY and SERPER_API_KEY:
    os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
    os.environ['SERPER_API_KEY'] = SERPER_API_KEY
    print("✅ API keys set")
else:
    print("🚨 API keys not set")

✅ API keys set


### Define the State Model for the Flow

The state model is like a shared notebook that all agents can read and write to.

It persists data throughout the entire workflow execution.

In [ ]:
from typing import Dict, Any
from pydantic import BaseModel

class ContentState(BaseModel):
  """
  State model that tracks information throughout the content creation workflow.

  Think of this as a form that gets filled out as the workflow progresses:
  - Start: Only URL is filled
  - After routing: Content type is determined
  - After processing: Final content is ready
  - Throughout: Metadata can be added by any step
  """
  url: str = "" # The source website we're analyzing
  content_type: str = ""  # "blog", "newsletter", or "linkedin"
  final_content: str = ""  # The final content after processing

**Why use state?**
1. **Data Persistence**: Information flows between different workflow steps
2. **Type Safety**: Pydantic ensures data integrity
3. **Flexibility**: Easy to add new fields as workflow evolves
4. **Debugging**: Clear view of what data exists at each step

### Create Specialized Agent Teams for Each Content Type

Think of these agents as factories for each content type:
- We're using factory functions to create specialized agent pairs
- Each content type gets its own **researcher + writer** team
- This allows for highly specialized expertise and output formats

1.   **Blog content creation agents.**




In [ ]:
from crewai import Agent, LLM
from crewai_tools import SerperDevTool

def create_blog_agents():
    """Create agents specialized for blog content"""

    blog_researcher = Agent(
        role="Blog Content Researcher",
        goal="Extract and analyze web content to identify key insights for blog posts",
        backstory="""You are an expert content researcher who specializes in analyzing
        web content and identifying the most valuable insights for creating engaging blog posts.
        You excel at understanding complex topics and breaking them down into digestible content.""",
        verbose=True,
        tool=[SerperDevTool()],
        llm=LLM(model="gpt-4o-mini"),
        max_iter=5
    )

    blog_writer = Agent(
        role="Blog Content Writer",
        goal="Transform research into engaging, well-structured blog posts",
        backstory="""You are a skilled blog writer with expertise in creating compelling content
        that engages readers and drives meaningful discussions. You excel at taking complex
        information and making it accessible and interesting.""",
        verbose=True,
        llm=LLM(model="gpt-4o-mini")
    )

    return blog_researcher, blog_writer

2.   **Newsletter content creation agents.**

In [ ]:
def create_newsletter_agents():
    """Create agents specialized for newsletter content"""

    newsletter_researcher = Agent(
        role="Newsletter Content Researcher",
        goal="Extract key insights from web content for newsletter format",
        backstory="""You are an expert at identifying the most newsworthy and actionable
        insights from web content. You understand what makes content valuable for newsletter
        subscribers and how to present information concisely.""",
        verbose=True,
        tool=[SerperDevTool()],
        llm=LLM(model="gpt-4o-mini"),
        max_iter=5
    )

    newsletter_writer = Agent(
        role="Newsletter Writer",
        goal="Create engaging newsletter content that provides immediate value",
        backstory="""You are a newsletter specialist who knows how to craft content that
        busy professionals want to read. You excel at creating scannable, actionable content
        with clear takeaways.""",
        verbose=True,
        llm=LLM(model="gpt-4o-mini")
    )

    return newsletter_researcher, newsletter_writer

3.   **LinkedIn content creation agents.**

In [ ]:
def create_linkedin_agents():
    """Create agents specialized for LinkedIn content"""

    linkedin_researcher = Agent(
        role="LinkedIn Content Researcher",
        goal="Extract professional insights suitable for LinkedIn audience",
        backstory="""You are an expert at identifying professional insights and industry
        trends that resonate with LinkedIn's professional audience. You understand what
        content drives engagement on professional networks.""",
        verbose=True,
        tool=[SerperDevTool()],
        llm=LLM(model="gpt-4o-mini"),
        max_iter=5
    )

    linkedin_writer = Agent(
        role="LinkedIn Content Writer",
        goal="Create engaging LinkedIn posts that drive professional engagement",
        backstory="""You are a LinkedIn content specialist who knows how to craft posts
        that get noticed in the professional feed. You excel at creating content that
        sparks meaningful professional discussions.""",
        verbose=True,
        llm=LLM(model="gpt-4o-mini")
    )

    return linkedin_researcher, linkedin_writer

### Define task creation functions for each content type

Task design principles:
1. Clear, specific instructions improve output quality
2. One-shot examples of the final output helps the agent accomplish better than without examples
3. Task guardrails is another advanced CrewAI feature that can be used to make the task outputs more deterministic. More on that [here](https://docs.crewai.com/en/concepts/tasks#task-guardrails).

1.   **Blog writing Task.**


In [ ]:
from crewai import Task

def create_blog_tasks(researcher, writer, url):
    """Create tasks for blog content generation"""

    research_task = Task(
        description=f"""
        Analyze the content from {url} and extract key insights for a blog post.
        Your analysis should identify:
        1. Main themes and key points
        2. Interesting insights or data points
        3. Potential angles for blog content
        4. Target audience considerations
        5. SEO-worthy topics and keywords

        Provide a comprehensive research summary that will guide blog writing.
        """,
        expected_output="A detailed research summary with key insights, themes, and recommendations for blog content",
        agent=researcher
    )

    writing_task = Task(
        description="""
        Create an engaging blog post based on the research findings.

        Requirements:
        - 800-1200 words
        - Engaging headline
        - Clear introduction with hook
        - Well-structured body with subheadings
        - Actionable insights or takeaways
        - Strong conclusion
        - SEO-optimized content
        - Professional yet accessible tone

        Format the output in markdown.
        """,
        expected_output="A complete, well-structured blog post in markdown format",
        agent=writer,
        context=[research_task]
    )

    return [research_task, writing_task]

2.   **Newsletter writing Task.**

In [ ]:
def create_newsletter_tasks(researcher, writer, url):
    """Create tasks for newsletter content generation"""

    research_task = Task(
        description=f"""
        Analyze the content from {url} and extract the most newsworthy insights for a newsletter.
        Focus on:
        1. Most important news or updates
        2. Actionable insights subscribers can use immediately
        3. Key statistics or data points
        4. Industry implications
        5. Quick takeaways for busy professionals

        Prioritize information that provides immediate value.
        """,
        expected_output="A focused research summary highlighting the most valuable and actionable insights",
        agent=researcher
    )

    writing_task = Task(
        description="""
        Create a compelling newsletter section based on the research.

        Requirements:
        - 400-600 words
        - Catchy subject line
        - Scannable format with bullet points
        - Clear action items or takeaways
        - Conversational yet professional tone
        - Include relevant links or resources
        - End with a clear call-to-action

        Format for easy reading in email.
        """,
        expected_output="A complete newsletter section with subject line and formatted content",
        agent=writer,
        context=[research_task]
    )

    return [research_task, writing_task]

2.   **LinkedIn post writing Task.**

In [ ]:
def create_linkedin_tasks(researcher, writer, url):
    """Create tasks for LinkedIn content generation"""

    research_task = Task(
        description=f"""
        Analyze the content from {url} and extract insights suitable for LinkedIn audience.
        Consider what would engage LinkedIn's professional audience based on the content.
        """,
        expected_output="Research summary focused on professional insights and engagement opportunities",
        agent=researcher
    )

    writing_task = Task(
        description="""
        Create an engaging LinkedIn post based on the research.

        Requirements:
        - 150-300 words (optimal LinkedIn length)
        - Professional yet conversational tone
        - Include relevant hashtags (3-5)
        - Pose a question to encourage engagement
        - Share a key insight or lesson learned from the content
        - Use line breaks for readability
        - Include a call-to-action for comments

        Make it shareable and discussion-worthy.
        """,
        expected_output="A complete LinkedIn post with hashtags and engagement elements",
        agent=writer,
        context=[research_task]
    )

    return [research_task, writing_task]

### **ContentRouterFlow Class Analysis**

This class orchestrates the entire content creation pipeline.

**Class Overview**

- `ContentRouterFlow` is a **subclass of `Flow[ContentState]`** that implements an event-driven workflow for dynamic content creation routing. It demonstrates CrewAI's Flow architecture with state management and specialized crew routing.

### Key Decorators & Their Roles

**💾 `@persist()`**

- **Purpose**: Enables automatic state persistence across workflow execution
- **Behavior**: Saves flow state to SQLite database by default
- **Benefits**: Workflow recovery, state tracking, and debugging capabilities
- **Scope**: Applied at class level to persist all method states

**🚀 `@start()`**
- **Purpose**: Marks the workflow entry point
- **Function**: Collects user input (URL + content type) and initializes state
- **Triggers**: Automatic when flow begins

**🧭 `@router(get_user_input)`**
- **Purpose**: Creates decision point for workflow routing
- **Logic**: Returns `self.state.content_type` to determine next path
- **Output**: Routes to "blog", "newsletter", or "linkedin" listeners

**👂 `@listen("event_name")`**
- **Purpose**: Creates event listeners for specific triggers
- **Behavior**: Methods execute when their specified event occurs
- **Examples**: `@listen("blog")`, `@listen("newsletter")`, `@listen("linkedin")`

## Workflow:

1. The workflow starts with **`@start()`** on **`get_user_input()`**.
2. The result of **`get_user_input()`** is routed by **`@router()`** to one of the `process_*_content` methods based on the content_type.
3. Whichever `process_*_content` method runs (e.g., `process_blog_content`), it will eventually complete and save the result to the flow's state.

In [ ]:
from crewai.flow.flow import Flow, listen, router, start
from crewai.flow.persistence import persist
from crewai import Crew

@persist(verbose=True)
class ContentRouterFlow(Flow[ContentState]):
    """
    A dynamic workflow that routes content creation to specialized crews.

    Flow Overview:
    1. START: Get user input (URL + content type)
    2. ROUTE: Direct to appropriate content crew
    3. PROCESS: Execute specialized content creation
    4. FINISH: Return the final content

    This flow demonstrates:
    - Event-driven architecture with decorators
    - State management across workflow steps
    - Dynamic routing based on user input
    - Parallel processing capabilities
    """

    @start()
    def get_user_input(self):
        """Get URL and desired content type from user"""
        # Enter your target URL here
        url = input("Enter your target URL: ") # example: https://blog.crewai.com/crewai-on-2025-ia-enablers-list-with-openai-and-anthropic/

        # Enter your content type: blog, newsletter, or LinkedIn
        content_type = input("Enter your desired content type (blog, newsletter, or linkedin): ")

        # Store in state
        self.state.url = url
        self.state.content_type = content_type

        return "Input collected"

    @router(get_user_input)
    def route_to_crew(self, previous_result):
        """Route to appropriate crew based on content type"""
        return self.state.content_type

    @listen("blog")
    def process_blog_content(self):
        """Process content using blog crew"""
        # Create blog agents
        researcher, writer = create_blog_agents()

        # Create blog tasks
        tasks = create_blog_tasks(researcher, writer, self.state.url)

        # Create and run crew
        blog_crew = Crew(
            agents=[researcher, writer],
            tasks=tasks,
            verbose=True
        )

        result = blog_crew.kickoff()
        self.state.final_content = result.raw

        return "Blog content created"

    @listen("newsletter")
    def process_newsletter_content(self):
        """Process content using newsletter crew"""

        # Create newsletter agents
        researcher, writer = create_newsletter_agents()

        # Create newsletter tasks
        tasks = create_newsletter_tasks(researcher, writer, self.state.url)

        # Create and run crew
        newsletter_crew = Crew(
            agents=[researcher, writer],
            tasks=tasks,
            verbose=True
        )

        result = newsletter_crew.kickoff()
        self.state.final_content = result.raw

        return "Newsletter content created"

    @listen("linkedin")
    def process_linkedin_content(self):
        """Process content using LinkedIn crew"""

        # Create LinkedIn agents
        researcher, writer = create_linkedin_agents()

        # Create LinkedIn tasks
        tasks = create_linkedin_tasks(researcher, writer, self.state.url)

        # Create and run crew
        linkedin_crew = Crew(
            agents=[researcher, writer],
            tasks=tasks,
            verbose=True
        )

        result = linkedin_crew.kickoff()
        self.state.final_content = result.raw

        return "LinkedIn content created"

### Run the Flow

Now we're ready to bring everything together into the flow and run it to start the automation. You should see a print out of the agent process so you can keep track of what the agents are doing.

In [ ]:
import asyncio

# Create and run the flow
flow = ContentRouterFlow()
result = await flow.kickoff_async()

╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: ContentRouterFlow                                                                                        │
│  ID: 05b562e5-1670-4d23-9033-71d72b3bcd63                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── Trace Batch Initialization ───────────────────────────────────────────╮
│ ✅ Trace batch initialized with session ID: 5e09c23e-b5c7-48e7-afd2-d0a81946df64                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

 Flow started with ID: 05b562e5-1670-4d23-9033-71d72b3bcd63

Output()

Enter your target URL: https://karpathy.bearblog.dev/animals-vs-ghosts/
Enter your desired content type (blog, newsletter, or linkedin): newsletter


...

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: b01b0cdc-3fdf-48e7-92a3-b1fe0d018680                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: **Subject Line: Transform Your Perspective on Intelligence with Insights from "Animals vs.       │
│  Ghosts"**                                                                                                      │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  Hello [Subscriber's Name],                                                                                     │
│                                                                                                                 │
│  In our rapidly evolving landscape of technology and artificial intelligence, redefining our understanding of   │
│  intelligence is paramount. Andrej Karpathy's recent article, "Animals vs. Ghosts," invites us to reconsider    │
│  how we interpret intelligence by exploring comparisons between biological organisms and the elusive realm of   │
│  ghosts. Here are the key takeaways and actionable insights that can provide immediate value to your            │
│  professional life.                                                                                             │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### 🗞️ **Important News Highlights**                                                                            │
│  - **Intelligence is Contextual**: The way we view intelligence can change dramatically based on the            │
│  comparisons we make. Karpathy's exploration of animals and ghosts challenges us to reflect on our definitions  │
│  and expectations of intelligence.                                                                              │
│                                                                                                                 │
│  ### 🚀 **Actionable Insights for Subscribers**                                                                 │
│  - **Broaden Your Perspective**:                                                                                │
│    - **What to Do**: Ask yourself how you define intelligence in various entities.                              │
│    - **Why It Matters**: Innovative approaches often stem from unconventional comparisons. Embrace the          │
│  complexity of both biological and artificial systems in your work.                                             │
│                                                                                                                 │
│  - **Model Comparisons**:                                                                                       │
│    - **What to Do**: Analyze different machine learning models focusing on their strengths and weaknesses.      │
│    - **Why It Matters**: A deeper understanding can re

Output()

 Saving flow state to memory for ID: 05b562e5-1670-4d23-9033-71d72b3bcd63


╭──────────────────────────────────────────────── Flow Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Flow Execution Completed                                                                                       │
│  Name: ContentRouterFlow                                                                                        │
│  ID: 05b562e5-1670-4d23-9033-71d72b3bcd63                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── Trace Batch Finalization ────────────────────────────────────────────╮
│ ✅ Trace batch finalized with session ID: 5e09c23e-b5c7-48e7-afd2-d0a81946df64. View here:                      │
│ https://app.crewai.com/crewai_plus/ephemeral_trace_batches/5e09c23e-b5c7-48e7-afd2-d0a81946df64?access_code=TRA │
│ CE-59ec52c21d , Access Code: TRACE-59ec52c21d                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Now that the run has completed, let's access the final content saved in our state via **`flow.state.final_content`**

In [ ]:
flow.state

StateWithId(id='05b562e5-1670-4d23-9033-71d72b3bcd63', url='https://karpathy.bearblog.dev/animals-vs-ghosts/', content_type='newsletter', final_content='**Subject Line: Transform Your Perspective on Intelligence with Insights from "Animals vs. Ghosts"**\n\n---\n\nHello [Subscriber\'s Name],\n\nIn our rapidly evolving landscape of technology and artificial intelligence, redefining our understanding of intelligence is paramount. Andrej Karpathy\'s recent article, "Animals vs. Ghosts," invites us to reconsider how we interpret intelligence by exploring comparisons between biological organisms and the elusive realm of ghosts. Here are the key takeaways and actionable insights that can provide immediate value to your professional life.\n\n---\n\n### 🗞️ **Important News Highlights**\n- **Intelligence is Contextual**: The way we view intelligence can change dramatically based on the comparisons we make. Karpathy\'s exploration of animals and ghosts challenges us to reflect on our definitions 

In [ ]:
from IPython.display import display, Markdown

# Display the actual content
display(Markdown("---"))

# Format the content nicely for notebook
content = str(flow.state.final_content)
display(Markdown(content))

---

**Subject Line: Transform Your Perspective on Intelligence with Insights from "Animals vs. Ghosts"**

---

Hello [Subscriber's Name],

In our rapidly evolving landscape of technology and artificial intelligence, redefining our understanding of intelligence is paramount. Andrej Karpathy's recent article, "Animals vs. Ghosts," invites us to reconsider how we interpret intelligence by exploring comparisons between biological organisms and the elusive realm of ghosts. Here are the key takeaways and actionable insights that can provide immediate value to your professional life.

---

### 🗞️ **Important News Highlights**
- **Intelligence is Contextual**: The way we view intelligence can change dramatically based on the comparisons we make. Karpathy's exploration of animals and ghosts challenges us to reflect on our definitions and expectations of intelligence.

### 🚀 **Actionable Insights for Subscribers**
- **Broaden Your Perspective**: 
  - **What to Do**: Ask yourself how you define intelligence in various entities.
  - **Why It Matters**: Innovative approaches often stem from unconventional comparisons. Embrace the complexity of both biological and artificial systems in your work.
  
- **Model Comparisons**: 
  - **What to Do**: Analyze different machine learning models focusing on their strengths and weaknesses.
  - **Why It Matters**: A deeper understanding can refine your applications, leading to enhanced outcomes in your projects.

### 📈 **Key Statistics/Data Points**
- While static data is sparsely mentioned in the article, the philosophical implications of intelligence metrics emerge. 
- **Action Item**: Seek out and pay attention to emerging metrics that challenge traditional norms of intelligence evaluation in your field.

### 🌍 **Industry Implications**
- The discourse around intelligence is not just academic; it has real-world consequences in AI development.
- **Adaptability is Key**: As industries pursue more intuitive AI, being able to evaluate various intelligence types becomes critical. This adaptability can prepare organizations for upcoming transitions as technology continues to evolve.

### 📋 **Quick Takeaways for Busy Professionals**
- **Rethink Intelligence Definitions**: Engage with and question how you currently perceive both biological and artificial intelligence.
- **Seek Multi-Faceted Approaches**: Incorporate insights from models across various fields to innovate and elevate your projects.
- **Stay Informed**: Keep an eye on evolving standards and metrics that redefine intelligence evaluation.

---

By fostering a nuanced understanding of intelligence, you’ll enhance your strategies in AI-driven environments, leading to more robust and adaptable solutions.

### 👥 **Call to Action**
Join the conversation! Reply to this email with your thoughts on how the comparison between animals and ghosts could reshape your view on intelligence, or share your own experiences in redefining intelligence within your work.

Best Regards,  
[Your Name]  
[Your Position]  
[Your Company]  

---

Stay ahead of the curve with us!

## **Next Steps**

- Explore CrewAI's advanced features like **`plot()`**, **`and_`**, and **`or_`** for complex logic. Access the [docs here](https://docs.crewai.com).
- Build human-in-the-loop workflows for interactive decision making
- Add guardrails + observability to your crews
- Scale to production with [CrewAI's enterprise](https://app.crewai.com) orchestration capabilities
- Join the [CrewAI community](https://community.crewai.com) and contribute to the ecosystem

**You're now equipped to build sophisticated, production-ready multi-agent AI systems that can handle complex, real-world automation challenges!**
